In [1]:
using Flight

In [82]:
ac = Cessna172Rv0(NED()) |> System #must use NED kinematics
trim_params = C172Rv0.TrimParameters(
    loc = LatLon(),
    h = HOrth(1000),
    ψ_nb = 0.0,
    TAS = 40.0,
    γ_wOb_n = 0.0,
    ψ_lb_dot = 0.00,
    θ_lb_dot = 0.0,
    β_a = 0.0,
    fuel = 0.5,
    mixture = 0.5,
    flaps = 0.0)

lm = linearize!(ac; trim_params)


LinearStateSpace{16, 4, 17, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(ψ = 1, θ = 2, φ = 3, ϕ = 4, λ = 5, h = 6, p = 7, q = 8, r = 9, v_x = 10, v_y = 11, v_z = 12, α_filt = 13, β_filt = 14, ω_eng = 15, fuel = 16)}}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(throttle = 1, aileron = 2, elevator = 3, rudder = 4)}}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(ψ = 1, θ = 2, φ = 3, ϕ = 4, λ = 5, h = 6, p = 7, q = 8, r = 9, TAS = 10, α = 11, β = 12, f_x = 13, f_y = 14, f_z = 15, ω_eng = 16, m_fuel = 17)}}}, ComponentArrays.ComponentMatrix{Float64, Matrix{Float64}, Tuple{ComponentArrays.Axis{(ψ = 1, θ = 2, φ = 3, ϕ = 4, λ = 5, h = 6, p = 7, q = 8, r = 9, v_x = 10, v_y = 11, v_z = 12, α_filt = 13, β_filt = 14, ω_eng = 15, fuel = 16)}, ComponentArrays.Axis{(ψ = 1, θ = 2, φ = 3, ϕ = 4, λ = 5, h = 6, p = 7, q = 8, r = 9, v_x = 10, v_y = 11, v_z = 12, α_filt = 13, β_fil

In [83]:
long_dyn = filter(lm;
    u = (:elevator, :throttle),
    x = (:v_x, :v_z, :θ, :q, :α_filt, :ω_eng),
    y = (:TAS, :α, :θ, :q, :f_x, :f_z, :ω_eng))

#could reduce the model further for CAS by omitting omega_eng and throttle

LinearStateSpace{6, 2, 7, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(v_x = 1, v_z = 2, θ = 3, q = 4, α_filt = 5, ω_eng = 6)}}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(elevator = 1, throttle = 2)}}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(TAS = 1, α = 2, θ = 3, q = 4, f_x = 5, f_z = 6, ω_eng = 7)}}}, ComponentArrays.ComponentMatrix{Float64, Matrix{Float64}, Tuple{ComponentArrays.Axis{(v_x = 1, v_z = 2, θ = 3, q = 4, α_filt = 5, ω_eng = 6)}, ComponentArrays.Axis{(v_x = 1, v_z = 2, θ = 3, q = 4, α_filt = 5, ω_eng = 6)}}}, ComponentArrays.ComponentMatrix{Float64, Matrix{Float64}, Tuple{ComponentArrays.Axis{(v_x = 1, v_z = 2, θ = 3, q = 4, α_filt = 5, ω_eng = 6)}, ComponentArrays.Axis{(elevator = 1, throttle = 2)}}}, ComponentArrays.ComponentMatrix{Float64, Matrix{Float64}, Tuple{ComponentArrays.Axis{(TAS = 1, α = 2, θ = 3, q = 4, f_x = 5, f_z = 6, ω_eng =

In [85]:
# world = SimpleWorld(Cessna172Rv0(LTF())) |> System; #for simulation, we use LTF kinematics

In [91]:
# #trim for the trim conditions above and simulate
# (success, result) = trim!(world; trim_params)
# @assert success
# sim = Simulation(world; t_end = 30, adaptive = true)
# Sim.run!(sim, verbose = true)
# # plots = make_plots(sim; Plotting.defaults...)
# plots = make_plots(TimeHistory(sim).ac.kinematics; Plotting.defaults...)
# save_plots(plots, save_folder = joinpath("tmp", "trim_sim_test"))

Simulation: Finished in 0.0092211 seconds


Saved figure tmp\trim_sim_test\common\01_e_nb.png


Saved figure tmp\trim_sim_test\common\02_Ob_geo.png


Saved figure tmp\trim_sim_test\common\03_Ob_xyh.png


Saved figure tmp\trim_sim_test\common\04_Ob_t3d.png


Saved figure tmp\trim_sim_test\common\05_ω_lb_b.png


Saved figure tmp\trim_sim_test\common\06_v_eOb_n.png


Saved figure tmp\trim_sim_test\common\07_v_eOb_b.png
